In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [3]:
%%sql

SELECT 
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [6]:
%%sql

SELECT
    g.name AS genre,
    COUNT(t.track_id) AS number_of_tracks
FROM genre AS g
LEFT JOIN track AS t ON t.genre_id = g.genre_id
GROUP BY 1
ORDER BY 2 DESC

Done.


genre,number_of_tracks
Rock,1297
Latin,579
Metal,374
Alternative & Punk,332
Jazz,130
TV Shows,93
Blues,81
Classical,74
Drama,64
R&B/Soul,61


In [7]:
%%sql

WITH 
    tracks_usa AS
               (
                SELECT
                    t.track_id,
                    t.genre_id,
                    g.name AS genre,
                    i.billing_country,
                    i.invoice_id,
                    t.album_id,
                    ar.name AS artist_name
                FROM track AS t
                INNER JOIN genre AS g ON g.genre_id = t.genre_id
                INNER JOIN album AS a ON a.album_id = t.album_id
                INNER JOIN artist AS ar ON ar.artist_id = a.artist_id
                INNER JOIN invoice_line AS il ON il.track_id = t.track_id
                INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
                WHERE i.billing_country = 'USA'
               )
SELECT 
    tusa.genre,
    COUNT(tusa.invoice_id) AS number_of_purchases,
    COUNT(tusa.invoice_id) / (
                             SELECT
                                 CAST(COUNT(invoice_id) AS FLOAT)
                             FROM tracks_usa 
                           ) * 100 AS number_of_purchases_per
FROM tracks_usa AS tusa
GROUP BY 1
ORDER BY 3 DESC

Done.


genre,number_of_purchases,number_of_purchases_per
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


The most popular genre purchased in the **USA** is the **Rock** genre. Out of track purchases in the US, it makes up **53%** of purchases. The other two popular genres are **Alternative & Punk** and **Metal**.

In [8]:
%%sql

WITH 
    tracks_usa AS
               (
                
                SELECT
                    t.track_id,
                    t.genre_id,
                    g.name AS genre,
                    i.billing_country,
                    i.invoice_id,
                    t.album_id,
                    a.title AS album_name,
                    ar.name AS artist_name
                FROM track AS t
                INNER JOIN genre AS g ON g.genre_id = t.genre_id
                INNER JOIN album AS a ON a.album_id = t.album_id
                INNER JOIN artist AS ar ON ar.artist_id = a.artist_id
                INNER JOIN invoice_line AS il ON il.track_id = t.track_id
                INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
                WHERE i.billing_country = 'USA'
               )
SELECT 
    tusa.artist_name AS artist_name,
    tusa.album_name AS album_name,
    tusa.genre AS genre,
    COUNT(tusa.invoice_id) AS number_of_purchases,
    COUNT(tusa.invoice_id) / (
                             SELECT
                                 CAST(COUNT(invoice_id) AS FLOAT)
                             FROM tracks_usa 
                           ) * 100 AS number_of_purchases_per
FROM tracks_usa AS tusa
WHERE genre = 'Metal'
GROUP BY 2
ORDER BY 4 DESC

Done.


artist_name,album_name,genre,number_of_purchases,number_of_purchases_per
System Of A Down,Mezmerize,Metal,22,2.093244529019981
Godsmack,Faceless,Metal,20,1.9029495718363463
Black Sabbath,Black Sabbath,Metal,18,1.7126546146527115
Guns N' Roses,Use Your Illusion II,Metal,9,0.8563273073263558
Judas Priest,Living After Midnight,Metal,8,0.7611798287345386
Apocalyptica,Plays Metallica By Four Cellos,Metal,8,0.7611798287345386
Black Sabbath,Black Sabbath Vol. 4 (Remaster),Metal,6,0.570884871550904
Motörhead,Ace Of Spades,Metal,5,0.47573739295908657
Mötley Crüe,Motley Crue Greatest Hits,Metal,5,0.47573739295908657
Metallica,Garage Inc. (Disc 2),Metal,3,0.285442435775452


The most popular artist purchased out of the rock genre is **Van Halen** with the album **The Best Of Van Halen, Vol. I**. The most purchased album in the genre is **Are You Experienced?** by **Jimi Hendrix**.

The artist with the most purchases in the **Alternative & Punk** is **Green Day** with the album **American Idiot**. The most purchased album is **Green** by **R.E.M.**

The artist with the most purchases in the **Metal** genre is **Black Sabbath** with the album **Black Sabbath**. The most purchased album is **Mezmerize** by **System Of A Down**.


In [9]:
%%sql

WITH employee_info AS 
                    (
                     SELECT 
                         c.customer_id,
                         c.support_rep_id,
                         c.first_name || ' ' || c.last_name AS customer_name,
                         e.employee_id,
                         e.hire_date,
                         e.first_name || ' ' || e.last_name AS employee_name
                     FROM customer AS c
                     INNER JOIN employee AS e ON e.employee_id = c.support_rep_id
                    )
SELECT
    ei.employee_name,
    ei.hire_date,
    COUNT(i.invoice_id) AS number_sales,
    CAST(SUM(i.total) AS float) AS total_sales
FROM employee_info AS ei
LEFT JOIN invoice AS i ON i.customer_id = ei.customer_id
GROUP BY 1

Done.


employee_name,hire_date,number_sales,total_sales
Jane Peacock,2017-04-01 00:00:00,212,1731.510000000004
Margaret Park,2017-05-03 00:00:00,214,1584.0000000000032
Steve Johnson,2017-10-17 00:00:00,188,1393.9200000000028


In [10]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


In [11]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.